# NCAA Tournament

## Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

from selenium import webdriver
from selenium.webdriver import Chrome
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests

import copy
import random

import multiprocessing
from joblib import Parallel, delayed

## Functions

In [2]:
#Webdriver parser used to extract the brackets
def parse2(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(url)
    sleep(3)
    sourceCode = driver.page_source
    return sourceCode

In [3]:
def score_bracket(bracket, results):
    points = 0
    for key in bracket.keys():
        if key != "points":
            bracket[key] = set(bracket[key])
            results[key] = set(results[key])
            points += rounds_points[key] * len(bracket[key] & results[key])
    return points

In [4]:
def comp_brackets(results, brackets, round_index=0, index=0):
	bracket_scores = [[0 for x in range(len(brackets))] for y in range(len(brackets))]
	if results["1"][0] != '':
		ordered = []
		results_copy = results.copy()
		for b in range(len(brackets)):
			bracket = brackets[b]
			score = score_bracket(bracket, results_copy)
			ordered.append((b, score))
		ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
		for x in range(len(ordered)):
			bracket_scores[ordered[x][0]][x] = 1
		return bracket_scores
	else:
		if index == int(rounds[round_index]) - 1:
			next_round_index = round_index + 1
			next_index = 0
		else:
			next_round_index = round_index
			next_index = index + 1


		if results[rounds[round_index]][index] == '':
			options = [results[rounds[round_index - 1]][index * 2], results[rounds[round_index - 1]][index * 2 + 1]]
			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[0]
			branch1 = comp_brackets(results_copy, brackets, next_round_index, next_index)

			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[1]
			branch2 = comp_brackets(results_copy, brackets, next_round_index, next_index)
			total_branch = np.add(branch1, branch2)
			return total_branch
		else:
			return comp_brackets(results, brackets, next_round_index, next_index)

def comp_brackets_weighted(results, brackets, team_to_seed, round_index=0, index=0, liklihood = 1):
	bracket_scores = [[0 for x in range(len(brackets))] for y in range(len(brackets))]
	if results["1"][0] != '':
		ordered = []
		results_copy = results.copy()
		for b in range(len(brackets)):
			bracket = brackets[b]
			score = score_bracket(bracket, results_copy)
			ordered.append((b, score))
		ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
		for x in range(len(ordered)):
			bracket_scores[ordered[x][0]][x] = 1 * liklihood
		return bracket_scores
	else:
		if index == int(rounds[round_index]) - 1:
			next_round_index = round_index + 1
			next_index = 0
		else:
			next_round_index = round_index
			next_index = index + 1


		if results[rounds[round_index]][index] == '':
			options = [results[rounds[round_index - 1]][index * 2], results[rounds[round_index - 1]][index * 2 + 1]]
			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[0]
			update = seed_vs_seed[team_to_seed[options[0]]][team_to_seed[options[1]]]
			branch1 = comp_brackets_weighted(results_copy, brackets, team_to_seed, next_round_index, next_index, liklihood * update)

			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[1]
			update = seed_vs_seed[team_to_seed[options[1]]][team_to_seed[options[0]]]
			branch2 = comp_brackets_weighted(results_copy, brackets, team_to_seed, next_round_index, next_index, liklihood * update)
			total_branch = np.add(branch1, branch2)
			return total_branch
		else:
			return comp_brackets_weighted(results, brackets, team_to_seed, next_round_index, next_index, liklihood)

In [5]:
def comp_MC(results, brackets, time_allocated = 10, samples = 1000, std_thresh = 0.1):
    start_time = time.time()
    trials = []
    while time.time() - start_time < time_allocated:
        bracket_scores = [np.array([0 for x in range(len(brackets))]) for y in range(len(brackets))]
        for trial in range(samples):
            results_copy = copy.deepcopy(results)
            for round_index in range(len(results_copy)-1):
                round = rounds[round_index]
                for index in range(len(results_copy[round])):
                    if results_copy[round][index] == '':
                        options = [results_copy[rounds[round_index - 1]][index * 2], results_copy[rounds[round_index - 1]][index * 2 + 1]]
                        option = options[random.choice([0,1])]
                        results_copy[round][index] = option
            ordered = []
            for b in range(len(brackets)):
                bracket = brackets[b]
                score = score_bracket(bracket, results_copy)
                ordered.append((b, score))
            ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
            for x in range(len(ordered)):
                bracket_scores[ordered[x][0]][x] += 1
        trials.append(np.array(bracket_scores) / samples)

        if len(trials) > 10:
            std = np.array(trials)
            test = std[0,:,0]
            max_win = np.argmax(test)
            std = std[:, max_win, 0]
            std = np.std(std) * 100
            print(std)
            if std < std_thresh:
                break
    trials = np.array(trials)
    std = trials.std(axis=0)
    trials = trials.mean(axis=0)
    return trials, std

def comp_MC_weighted(results, brackets, team_to_seed, time_allocated = 10, samples = 1000, std_thresh = 0.1):
    start_time = time.time()
    trials = []
    while time.time() - start_time < time_allocated:
        bracket_scores = [np.array([0 for x in range(len(brackets))]) for y in range(len(brackets))]
        for trial in range(samples):
            results_copy = copy.deepcopy(results)
            for round_index in range(len(results_copy)-1):
                round = rounds[round_index]
                for index in range(len(results_copy[round])):
                    if results_copy[round][index] == '':
                        options = [results_copy[rounds[round_index - 1]][index * 2], results_copy[rounds[round_index - 1]][index * 2 + 1]]
                        weights = [seed_vs_seed[team_to_seed[options[0]]][team_to_seed[options[1]]], seed_vs_seed[team_to_seed[options[1]]][team_to_seed[options[0]]]]
                        option = options[random.choices([0,1], weights=weights)[0]]
                        results_copy[round][index] = option
            ordered = []
            for b in range(len(brackets)):
                bracket = brackets[b]
                score = score_bracket(bracket, results_copy)
                ordered.append((b, score))
            ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
            for x in range(len(ordered)):
                bracket_scores[ordered[x][0]][x] += 1
        trials.append(np.array(bracket_scores) / samples)
        if len(trials) > 10:
            std = np.array(trials)
            test = std[0,:,0]
            max_win = np.argmax(test)
            std = std[:, max_win, 0]
            std = np.std(std) * 100
            print(std)
            if std < std_thresh:
                break
    trials = np.array(trials)
    std = trials.std(axis=0)
    trials = trials.mean(axis=0)
    return trials, std

In [6]:
#Gets the actual games and results that have been played
def get_so_far(url):
    print(url)
    bracket = {"64": [], "32": [], "16": [], "8": [], "4": [], "2": [], "1": []}
    data = requests.get(url)
    soup = BeautifulSoup(data.text, features='html.parser')
    index = 0
    team_to_seed = {}
    for x in range(1, 64):
        class_i = "matchup m_" + str(x)
        mydivs = soup.find_all("div", {"class": class_i})
        if mydivs == []:
            class_ii = class_i + " userPickable"
            mydivs = soup.find_all("div", {"class": class_ii})
            if mydivs == []:
                class_ii = class_i + " homeOnBottom" + " userPickable"
                mydivs = soup.find_all("div", {"class": class_ii})
            mydivs = mydivs[0].find_all("span", {"class": "name"})
            team1 = mydivs[0].text.strip()
            team2 = mydivs[1].text.strip()
            team3 = mydivs[2].text.strip()
            team4 = mydivs[3].text.strip()
        else:
            mydivs = mydivs[0].find_all("div", {"class": "slots"})
            team1 = mydivs[0].find_all("div", {"class": "slot s_1"})
            team1 = team1[0].find_all("span", {"class": "name"})
            team1 = team1[0].text.strip()

            team2 = mydivs[0].find_all("div", {"class": "slot s_2"})
            team2 = team2[0].find_all("span", {"class": "name"})
            team2 = team2[0].text.strip()
        if x <= 32:
            bracket["64"].append(team1)
            bracket["64"].append(team2)
            team_to_seed[team1] = seed_order[index]
            team_to_seed[team2] = seed_order[index + 1]
            index += 2
            if index == 16:
                index = 0
        elif x <= 48:
            bracket["32"].append(team1)
            bracket["32"].append(team3)
        elif x <= 56:
            bracket["16"].append(team1)
            bracket["16"].append(team3)
        elif x <= 60:
            bracket["8"].append(team1)
            bracket["8"].append(team3)
        elif x <= 62:
            bracket["4"].append(team1)
            bracket["4"].append(team3)
        elif x <= 63:
            bracket["2"].append(team1)
            bracket["2"].append(team3)

    mydivs = soup.find_all("div", {"class": "slot userPickable"})
    if len(mydivs) == 0:
        mydivs = soup.find_all("span", {"class": "actual winner"})
    teams = mydivs[0].find_all("span", {"class": "name"})
    team1 = teams[0].text.strip()
    # team2 = teams[1].text.strip()
    bracket["1"].append(team1)
    bracket["points"] = 1920
    return bracket, team_to_seed

#Gets the picks for a bracket for each game
def get_bracket(url):
    bracket = {"64": [], "32": [], "16": [], "8": [], "4": [], "2": [], "1": [], "points": 0}
    data = requests.get(url)
    soup = BeautifulSoup(data.text, features='html.parser')
    classes = []
    for x in range(1, 64):
        class_i = "matchup m_" + str(x)

        mydivs = soup.find_all("div", {"class": class_i})

        if mydivs == []:
            class_ii = class_i + " userPickable"
            mydivs = soup.find_all("div", {"class": class_ii})

            if mydivs == []:
                class_ii = class_i + " homeOnBottom" + " userPickable"
                mydivs = soup.find_all("div", {"class": class_ii})
            try:
                mydivs = mydivs[0].find_all("span", {"class": "name"})
            except:
                return {'64': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '32': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '16': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '8': ['1', '1', '1', '1', '1', '1', '1', '1'], '4': ['1', '1', '1', '1'], '2': ['1', '1'],
                        '1': ['1'], 'points': 260}

            team1 = mydivs[0].text.strip()
            team2 = mydivs[1].text.strip()
            team3 = mydivs[2].text.strip()
            team4 = mydivs[3].text.strip()
        else:
            mydivs = mydivs[0].find_all("div", {"class": "slots"})
            team1 = mydivs[0].find_all("div", {"class": "slot s_1"})
            team1 = team1[0].find_all("span", {"class": "name"})
            team1 = team1[0].text.strip()

            team2 = mydivs[0].find_all("div", {"class": "slot s_2"})
            team2 = team2[0].find_all("span", {"class": "name"})
            team2 = team2[0].text.strip()

        if x <= 32:
            bracket["64"].append(team1)
            bracket["64"].append(team2)
        elif x <= 48:
            bracket["32"].append(team2)
            bracket["32"].append(team4)
            if team1 == team2:
                bracket["points"] += 10
            if team3 == team4:
                bracket["points"] += 10
        elif x <= 56:
            bracket["16"].append(team2)
            bracket["16"].append(team4)
            if team1 == team2:
                bracket["points"] += 20
            if team3 == team4:
                bracket["points"] += 20
        elif x <= 60:
            bracket["8"].append(team2)
            bracket["8"].append(team4)
            if team1 == team2:
                bracket["points"] += 40
            if team3 == team4:
                bracket["points"] += 40
        elif x <= 62:
            bracket["4"].append(team2)
            bracket["4"].append(team4)
            if team1 == team2:
                bracket["points"] += 80
            if team3 == team4:
                bracket["points"] += 80
        elif x <= 63:
            bracket["2"].append(team2)
            bracket["2"].append(team4)
            if team1 == team2:
                bracket["points"] += 160
            if team3 == team4:
                bracket["points"] += 160
    try:
        mydivs = soup.find_all("div", {"class": "slot userPickable"})
        teams = mydivs[0].find_all("span", {"class": "name"})
    except:
        try:
            mydivs = soup.find_all("div", {"class": "slot wWinner userPickable"})
            teams = mydivs[0].find_all("span", {"class": "name"})
        except:
            mydivs = soup.find_all("div", {"class": "slot s_2 correct wWinner userPickable"})
            teams = mydivs[0].find_all("span", {"class": "name"})
    team1 = teams[0].text.strip()
    team2 = teams[1].text.strip()
    bracket["1"].append(team2)
    if team1 == team2:
        bracket["points"] += 320

    return bracket

#Gets the IDs of all the participants in a group
def get_group(group):
    IDs = []
    soup = BeautifulSoup(parse2(group))
    mydivs = soup.find_all("table", {"class": "type_entries"})
    entries = mydivs[0].find_all("a", {"class": "entry"})
    for entry in entries:
        IDs.append((entry.get('href'), entry.text.strip()))
    return IDs

## Constants

In [7]:
base_bracket_url = "https://fantasy.espn.com/tournament-challenge-bracket/2023/en/"

rounds = ["64", "32", "16", "8", "4", "2", "1"]

rounds_points = {"64": 0, "32": 10, "16": 20, "8": 40, "4": 80, "2": 160, "1": 320}

seed_vs_seed = {'1': {'1': 0.5, '2': 0.539, '3': 0.634, '4': 0.711, '5': 0.839, '6': 0.706, '7': 0.857, '8': 0.798, '9': 0.901, '10': 0.857, '11': 0.556, '12': 1.0, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.993}, '2': {'1': 0.461, '2': 0.5, '3': 0.603, '4': 0.444, '5': 0.167, '6': 0.722, '7': 0.694, '8': 0.444, '9': 0.5, '10': 0.645, '11': 0.833, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.938, '16': 0.5}, '3': {'1': 0.366, '2': 0.397, '3': 0.5, '4': 0.625, '5': 0.5, '6': 0.576, '7': 0.611, '8': 0.5, '9': 0.5, '10': 0.692, '11': 0.679, '12': 0.5, '13': 0.5, '14': 0.847, '15': 0.5, '16': 0.5}, '4': {'1': 0.289, '2': 0.556, '3': 0.375, '4': 0.5, '5': 0.563, '6': 0.333, '7': 0.333, '8': 0.364, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.69, '13': 0.79, '14': 0.5, '15': 0.5, '16': 0.5}, '5': {'1': 0.161, '2': 0.833, '3': 0.5, '4': 0.438, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.671, '13': 0.842, '14': 0.5, '15': 0.5, '16': 0.5}, '6': {'1': 0.294, '2': 0.278, '3': 0.424, '4': 0.667, '5': 0.5, '6': 0.5, '7': 0.625, '8': 0.5, '9': 0.5, '10': 0.6, '11': 0.634, '12': 0.5, '13': 0.5, '14': 0.875, '15': 0.5, '16': 0.5}, '7': {'1': 0.143, '2': 0.306, '3': 0.389, '4': 0.667, '5': 0.5, '6': 0.333, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.601, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '8': {'1': 0.202, '2': 0.556, '3': 0.5, '4': 0.636, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.518, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '9': {'1': 0.099, '2': 0.5, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.482, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '10': {'1': 0.143, '2': 0.355, '3': 0.308, '4': 0.5, '5': 0.5, '6': 0.4, '7': 0.399, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '11': {'1': 0.444, '2': 0.167, '3': 0.321, '4': 0.5, '5': 0.5, '6': 0.366, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.9, '15': 0.5, '16': 0.5}, '12': {'1': 0.1, '2': 0.5, '3': 0.5, '4': 0.31, '5': 0.329, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.75, '14': 0.5, '15': 0.5, '16': 0.5}, '13': {'1': 0.5, '2': 0.5, '3': 0.5, '4': 0.21, '5': 0.158, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.25, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '14': {'1': 0.5, '2': 0.5, '3': 0.153, '4': 0.5, '5': 0.5, '6': 0.125, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.1, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '15': {'1': 0.5, '2': 0.063, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '16': {'1': 0.007, '2': 0.5, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}}

seed_order = ['1', '16', "8", "9", "5", "12", "4", "13", "6", "11", "3", "14", "7", "10", "2", "15"]

## Main

In [11]:
# This is the only thing you need to change. Just change to the url of your group
group_url = "https://fantasy.espn.com/tournament-challenge-bracket/2023/en/group?groupID=5270338"

IDs = get_group(group_url)

results, team_to_seed = get_so_far(base_bracket_url + IDs[0][0])

brackets = []
for x in tqdm(range(len(IDs))):
    bracket = get_bracket(base_bracket_url + IDs[x][0])
    brackets.append(bracket)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\aidan\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\aidan\AppData\Local\Temp\ipykernel_25612\4167765881.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


https://fantasy.espn.com/tournament-challenge-bracket/2023/en/entry?entryID=76523085


100%|██████████| 17/17 [00:23<00:00,  1.36s/it]


In [25]:
results['1'] = ['']

In [26]:
# Runs weighted Monte Carlo Simulation.
# Weights are proportional to how likely that bracket is to happen based on historical matchups.

# time_allocated = how long you want to run the code for before getting results as Monte Carlo is an approximation
# std_thresh = standard deviation of the win probabilities that you want to hit. If it gets under this threshold before the 
# time threshold is hit. The program will break. 

total, std = comp_MC_weighted(results, brackets, team_to_seed, samples = 1000, time_allocated=10, std_thresh=0.1)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values(list(df.columns.values), ascending=False)
df.to_csv('Results/2023/DadAlwaysWins_weighted_elite8.csv')
df

1.4708180666295736
1.510978343840687
1.4528486606557551
1.4283749864936752
1.3935406544322801
1.3523128336298493
1.3119889019701068
1.3018149723452312
1.2716817664948454
1.4284519592901939
1.399773224263514
1.4109664936776443
1.3800769820450252
1.352466881911221
1.3617283135780036
1.353917174710597
1.3287988738661403
1.3183748499186236
1.3248561270687556
1.327675495828034
1.3116543160242529
1.2912226219265963


,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th,14th,15th,16th,17th
CreightonWaters,80.065625,6.646875,11.756250,1.531250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.0000,0.0000,0.0
El Jefe,7.687500,22.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.5750,0.000000,0.00000,37.5875,0.0000,0.0
Ciao Papa,6.646875,42.768750,40.981250,8.090625,1.512500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.0000,0.0000,0.0
Kate_Gilson_3,5.600000,5.525000,1.775000,0.000000,0.000000,2.103125,1.968750,0.000000,0.000000,57.262500,25.765625,0.0000,0.000000,0.00000,0.0000,0.0000,0.0
LalaDon'tCare,0.000000,17.309375,0.000000,30.059375,14.728125,19.743750,13.006250,3.050000,2.103125,0.000000,0.000000,0.0000,0.000000,0.00000,0.0000,0.0000,0.0
JMGilly 1,0.000000,5.600000,3.759375,1.765625,1.775000,0.000000,0.000000,2.103125,1.968750,0.000000,0.000000,0.0000,31.109375,51.91875,0.0000,0.0000,0.0
Aidansg 1,0.000000,0.000000,41.728125,45.371875,9.612500,3.287500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.0000,0.0000,0.0
Amelia,0.000000,0.000000,0.000000,13.181250,28.081250,17.778125,34.481250,6.478125,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.0000,0.0000,0.0
William Steenbergen 1,0.000000,0.000000,0.000000,0.000000,31.109375,0.000000,26.153125,25.765625,0.000000,0.000000,0.000000,0.0000,6.478125,10.49375,0.0000,0.0000,0.0
LeBrontosaurus,0.000000,0.000000,0.000000,0.000000,13.181250,25.978125,17.912500,5.340625,0.000000,0.000000,0.000000,37.5875,0.000000,0.00000,0.0000,0.0000,0.0


In [12]:
# Unweighted Monted Carlo Bracket. Means all possible brackets are equally likely (1 beating a 16 is the same odds as 16 beating a 1)
total, std = comp_MC(results, brackets, samples = 100000, time_allocated=300, std_thresh=0.1)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df.to_csv('Results/2023/DadAlwaysWins_begin_unweighted_sweet16.csv')
df

0.1111791793338815
0.10660545118436593


,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th,14th,15th,16th,17th
CreightonWaters,24.797667,10.605500,7.345667,7.243833,5.540833,3.803750,5.055917,5.462000,3.996417,7.657333,4.458750,4.395667,4.284500,3.414917,1.492083,0.445167,0.000000
LalaDon'tCare,24.593667,16.563917,12.689250,9.698333,6.693833,5.570083,5.175583,5.348417,4.643000,4.199500,2.567083,1.987417,0.256917,0.013000,0.000000,0.000000,0.000000
Amelia,7.176000,13.675417,14.070333,11.367833,9.784583,9.203500,8.147083,5.120833,5.042000,5.054750,2.568750,3.860917,3.028500,1.513833,0.385667,0.000000,0.000000
Nota Pene,6.719167,2.099250,1.749167,1.772500,1.469000,1.353750,1.791500,2.449167,2.987000,3.878250,8.749000,9.627000,7.970667,9.786500,7.062917,11.364667,19.170500
TGilly,6.261000,0.423833,1.682083,3.007333,3.036833,3.516167,5.320250,9.108750,9.266333,7.191667,5.975667,5.794167,6.309583,8.245333,14.655750,10.205250,0.000000
Cgilly61 2,5.950583,11.367667,10.184083,7.137167,5.848250,5.359500,4.658667,3.885917,4.357417,5.973833,7.799583,9.071250,8.687667,5.940833,1.836833,1.742667,0.198083
William Steenbergen 1,5.102917,7.700083,8.125583,7.837000,6.977583,4.115000,4.626500,4.210083,4.744500,4.780250,4.900583,6.170667,9.026250,8.435500,7.760083,4.522417,0.965000
Kate_Gilson_3,4.703167,4.774833,3.141833,2.535167,2.495083,3.326500,4.006750,3.249583,3.107083,3.770083,4.755667,5.284250,8.053667,7.382833,10.283333,10.564750,18.565417
Ciao Papa,3.731667,8.842917,12.400667,11.789083,11.734500,9.118500,6.666917,9.490833,6.086750,5.469000,5.418750,3.379750,2.396917,2.263083,1.210667,0.000000,0.000000
JMGilly 1,3.627083,5.226500,2.748333,2.213000,1.316250,1.480833,1.975333,2.012917,2.437000,1.544250,2.254417,5.316250,7.317917,11.404250,19.423583,14.151500,15.550583


In [32]:
# Runs weighted Simulation.
# Weights are proportional to how likely that bracket is to happen based on historical matchups.
# Checks all possible bracket permutations remaining.
# Takes too long before ~Elite 8 or most of the was through sweet 16 as it is O(2^n)
total = comp_brackets_weighted(results, brackets, team_to_seed)
# print(total)
total = total * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values(list(df.columns.values), ascending=False)
df

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th,14th,15th,16th,17th
CreightonWaters,80.313481,6.507812,11.641758,1.573453,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
El Jefe,7.543734,22.631203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.325063,0.000000,0.000000,37.536505,0.000000,0.000000
Ciao Papa,6.507812,42.705937,41.580870,7.851609,1.390275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Kate_Gilson_3,5.671477,5.144731,1.740975,0.000000,0.000000,2.071844,2.071844,0.000000,0.000000,57.304384,26.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LalaDon'tCare,0.000000,17.375344,0.000000,30.044665,15.087469,19.294652,13.015625,3.146906,2.071844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
JMGilly 1,0.000000,5.671477,3.402886,1.741845,1.740975,0.000000,0.000000,2.071844,2.071844,0.000000,0.000000,0.000000,31.273134,52.062500,0.000000,0.000000,0.000000
Aidansg 1,0.000000,0.000000,41.670015,45.809306,9.425933,3.131250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Amelia,0.000000,0.000000,0.000000,13.015625,28.103094,18.234375,34.420040,6.263370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
William Steenbergen 1,0.000000,0.000000,0.000000,0.000000,31.273134,0.000000,26.031250,26.031250,0.000000,0.000000,0.000000,0.000000,6.263370,10.437500,0.000000,0.000000,0.000000
LeBrontosaurus,0.000000,0.000000,0.000000,0.000000,13.015625,26.031250,18.234375,5.218750,0.000000,0.000000,0.000000,37.536505,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
df.to_csv('Results/2023/DadAlwaysWins_weighted_elite8.csv')

In [15]:
# Runs unweighted Simulation.
# Weights are proportional to how likely that bracket is to happen based on historical matchups.
# Checks all possible bracket permutations remaining.
# Takes too long before ~Elite 8 or most of the was through sweet 16 as it is O(2^n)
total = comp_brackets(results, brackets)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th,14th,15th,16th,17th
CreightonWaters,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LeBrontosaurus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TGilly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
El Jefe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
Cgilly61 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
BrutustheWalnut,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
William Steenbergen 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
JMGilly 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
georgia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ciao Papa,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
